In [1]:
%load_ext beam_setup

Setting up the Beam environment for interactive use
Standard modules will be automatically imported so you can use them without explicit import
Done importing packages. It took:  3.3 seconds
Beam library is loaded from path: /home/elad/docker/beamds/src/beam
The Beam version is: 2.3.7


# LLM Tools for function calling

Lets start by defining a function with its description:

In [20]:
import yfinance as yf

msft = yf.Ticker("MSFT")

# get all stock info
msft.info

# get historical market data
hist = msft.history(period="1mo")

def ticker_history(ticker, period):
    """
    Get historical stock market data
    """
    ticker = yf.Ticker(ticker.upper())
    return ticker.history(period=period)

define an LLMTool based on the given function

In [21]:
from beam.llm import LLMTool, LLMToolProperty

In [22]:
ticker = LLMToolProperty('ticker', 'str', 'The ticker of the stock (should be upper cased)', required=True)
period = LLMToolProperty('period', 'str', 'The historical period to get data for, in the form [1mo, 2mo, 10d, 2y, etc..]', required=True)

In [23]:
tool = LLMTool(func=ticker_history, ticker=ticker, period=period)

Lets access an llm resouce and add the tool for its tools list

In [24]:
llm = resource('openai:///gpt-4')

In [25]:
llm.add_tool(tool)

ask a question

In [26]:
res = llm.ask('give me the last two month stock data of broadcom')

In [27]:
res.text

" and ibm.\n\n    >>> from stock import Stock\n    >>> s1 = Stock('BRCM')\n    >>> s2 = Stock('IBM')\n    >>> s1.get_last_two_month_data()\n    >>> s2.get_last_two_month_data()\n\n\n### 2. get the last two month stock data of broadcom and ibm.\n\n    >>> from stock import Stock\n    >>> s1 = Stock('BRCM')\n    >>> s2 = Stock('IBM')\n    >>> s1.get_"

In [10]:
et = tool(res)

In [11]:
et.response.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-10-13 00:00:00-04:00,909.000000,925.909973,882.090027,883.179993,2831200,0.0,0.0
2023-10-16 00:00:00-04:00,890.969971,915.789978,890.969971,902.570007,2115000,0.0,0.0
2023-10-17 00:00:00-04:00,885.150024,892.979980,866.000000,884.400024,3300800,0.0,0.0
2023-10-18 00:00:00-04:00,879.950012,895.630005,875.200012,887.030029,2040400,0.0,0.0
2023-10-19 00:00:00-04:00,887.020020,889.489990,865.669983,867.830017,2662000,0.0,0.0


if the docstring in our function is informative enough, we can omit the parameters description

In [19]:
def ticker_history(ticker, period):
    """
    Get historical stock market data

    Args:
        ticker (str): The ticker of the stock (should be upper cased).
        period (int): The historical period to get data for, in the form [1mo, 2mo, 10d, 2y, etc..].
    
    Returns:
        data: A pandas DataFrame of the stock market data.
    """
    ticker = yf.Ticker(ticker.upper())
    return ticker.history(period=period)

In [13]:
tool = LLMTool(func=ticker_history)

In [14]:
llm = resource('openai:///gpt-4')

In [15]:
llm.add_tool(tool)

In [16]:
res = llm.ask('What was the top price of apple last year?')

In [17]:
et = tool(res)

In [21]:
et.args

['AAPL', '1y']

In [48]:
et.args

['AAPL', '1y']

In [19]:
et.response.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-12-13 00:00:00-05:00,148.669021,149.136410,143.438264,144.661423,93886200,0.0,0.0
2022-12-14 00:00:00-05:00,144.542091,145.844807,140.375378,142.413986,82291200,0.0,0.0
2022-12-15 00:00:00-05:00,140.325648,141.011815,135.273883,135.741272,98931900,0.0,0.0
2022-12-16 00:00:00-05:00,135.930220,136.884875,132.986666,133.762329,160156900,0.0,0.0
2022-12-19 00:00:00-05:00,134.358976,134.448472,130.590050,131.634201,79592600,0.0,0.0


Now lets use the openai simulator to achive the same effect

In [22]:
from beam.llm import openai_simulator as openai

In [25]:
res = openai.ChatCompletion.create(model='openai:///gpt-4', messages=[{"role": "user", "content": "give me the last two month stock data of broadcom"}],
    tools = [
        {
            "type": "function",
            "function": {
                "name": "ticker_history",
                "description": "Get historical stock market data",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker": {"type": "string", "description": "The ticker of the stock (should be upper cased)."},
                        "period": {"type": "string","description": "The historical period to get data for, in the form [1mo, 2mo, 10d, 2y, etc..]."},
                    },
                    "required": ["ticker", "period"],
                },
            },
        }
    ])

2023-12-13 12:58:35 | BeamLog | WARNING | finish_reason is tool_calls


In [39]:
res.choices[0].message

<OpenAIObject at 0x7fd70d19a1b0> JSON: {
  "role": "assistant",
  "content": null,
  "tool_calls": [
    {
      "id": "call_QVvITgTRcfCZUoLyAJfcwxbA",
      "type": "function",
      "function": {
        "name": "ticker_history",
        "arguments": "{\n  \"ticker\": \"AVGO\",\n  \"period\": \"2mo\"\n}"
      }
    }
  ]
}

In [45]:
et.tool

{}

In [2]:
from beam.llm import LLMTask

In [3]:
task = LLMTask('random_names_generator', system='you like names that start with R', 
               description='Generate a list of random names for babies', 
               input_kwargs={'num': 'number of names to generate', 'geneder': 'either boy or girl'}, output_kwargs={'res': 'a list of generated names'},
               llm='openai:///gpt-4')

In [4]:
res = task(num=3, gender='boy')
res.task_result

{'res': ['Ryan', 'Riley', 'Raphael']}

In [5]:
res = task(num=4, gender='girl')
res.task_result

{'res': ['Rachel', 'Rebecca', 'Rosalie', 'Ruth']}